In [1]:
import sympy as sym



In [2]:
a, b, c, x, tau = sym.symbols('a b c x tau', real=True)



In [3]:
f = {
    'const':a,
    'linear':a * x + b,
    'quadratic':a * x ** 2 + b * x + c,
    'sinusoidal':a * sym.sin(x*b + c),
    'exp':a * sym.exp(-x*b + c)
}


In [4]:
for name, load in f.items():
    print('load '.ljust(40, '='))
    print(load)
    print(sym.pycode(load))

    print('sfd '.ljust(40, '-'))
    sfd = sym.integrate(load.subs({x:tau}), (tau, 0, x))
    print(sfd)
    print(sym.pycode(sfd))

    print('bmd '.ljust(40, '-'))
    bmd = sym.integrate(sfd.subs({x:tau}), (tau, 0, x))
    print(bmd)
    print(sym.pycode(bmd))
    



load ===================================
a
a
sfd ------------------------------------
a*x
a*x
bmd ------------------------------------
a*x**2/2
(1/2)*a*x**2
load ===================================
a*x + b
a*x + b
sfd ------------------------------------
a*x**2/2 + b*x
(1/2)*a*x**2 + b*x
bmd ------------------------------------
a*x**3/6 + b*x**2/2
(1/6)*a*x**3 + (1/2)*b*x**2
load ===================================
a*x**2 + b*x + c
a*x**2 + b*x + c
sfd ------------------------------------
a*x**3/3 + b*x**2/2 + c*x
(1/3)*a*x**3 + (1/2)*b*x**2 + c*x
bmd ------------------------------------
a*x**4/12 + b*x**3/6 + c*x**2/2
(1/12)*a*x**4 + (1/6)*b*x**3 + (1/2)*c*x**2
load ===================================
a*sin(b*x + c)
a*math.sin(b*x + c)
sfd ------------------------------------
Piecewise((a*cos(c)/b - a*cos(b*x + c)/b, Ne(b, 0)), (a*x*sin(c), True))
((a*math.cos(c)/b - a*math.cos(b*x + c)/b) if (b != 0) else (a*x*math.sin(c)))
bmd ------------------------------------
Piecewise((a*x*cos(

In [7]:
load = f['sinusoidal']
sfd = sym.integrate(load.subs({x:tau}), (tau, 0, x))
sfd

Piecewise((a*cos(c)/b - a*cos(b*x + c)/b, Ne(b, 0)), (a*x*sin(c), True))

In [8]:
bmd = sym.integrate(sfd.subs({x:tau}), (tau, 0, x))
bmd

Piecewise((a*x*cos(c)/b + a*sin(c)/b**2 - a*sin(b*x + c)/b**2, Ne(b, 0)), (a*x**2*sin(c)/2, True))

In [5]:
for name, load in f.items():
    print('@pytest.fixture')
    print(f'def gen_case_{name}(a, b, c):')
    print('    # load '.ljust(40, '-'))
    print('    def load(x):')
    print('        return', sym.pycode(load).replace('math.', 'np.'))

    print('    # sfd '.ljust(40, '-'))
    sfd = sym.integrate(load.subs({x:tau}), (tau, 0, x))
    print('    def sfd(x):')
    print('        return', sym.pycode(sfd).replace('math.', 'np.'))

    print('    # bmd '.ljust(40, '-'))
    bmd = sym.integrate(sfd.subs({x:tau}), (tau, 0, x))
    print('    def bmd(x):')
    print('        return', sym.pycode(bmd).replace('math.', 'np.'))

    print('    return load, sfd, bmd\n\n')



@pytest.fixture
def gen_case_const(a, b, c):
    # load -----------------------------
    def load(x):
        return a
    # sfd ------------------------------
    def sfd(x):
        return a*x
    # bmd ------------------------------
    def bmd(x):
        return (1/2)*a*x**2
    return load, sfd, bmd


@pytest.fixture
def gen_case_linear(a, b, c):
    # load -----------------------------
    def load(x):
        return a*x + b
    # sfd ------------------------------
    def sfd(x):
        return (1/2)*a*x**2 + b*x
    # bmd ------------------------------
    def bmd(x):
        return (1/6)*a*x**3 + (1/2)*b*x**2
    return load, sfd, bmd


@pytest.fixture
def gen_case_quadratic(a, b, c):
    # load -----------------------------
    def load(x):
        return a*x**2 + b*x + c
    # sfd ------------------------------
    def sfd(x):
        return (1/3)*a*x**3 + (1/2)*b*x**2 + c*x
    # bmd ------------------------------
    def bmd(x):
        return (1/12)*a*x**4 + (1/6)*b*x**3